In [ ]:
import numpy as np
import mayfly as mf
import h5py
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import os 
import sys
import json
import scipy.signal
import scipy.stats
import scipy.interpolate
import pickle as pkl
import torch
import scipy.optimize

PATH = '/storage/home/adz6/group/project/'
RESULTPATH = os.path.join(PATH, 'results/mayfly')
PLOTPATH = os.path.join(PATH, 'plots/mayfly')
DATAPATH = os.path.join(PATH, 'datasets/data')

def GetScoreFiles(path2scores, params, random=True):
    
    radii = params['r']
    angles = params['pa']
    nsample = params['N']
    
    score_file_list = []
    for i, file in enumerate(os.listdir(path2scores)):
        
        if not os.path.isdir(os.path.join(path2scores, file)):
            continue
        
        try:
            file_angle = float(file.split('grid_')[-1].split('_')[0])
            file_rad = int(file.split('cm')[0].split('_')[-1])
            file_has_random = file.find('random')
        except BaseException as err:
            print(err, file)
            continue

        for j, item in enumerate(zip(radii, angles, nsample)):
            #print(item, file_rad, file_angle, file_has_random, file)
            
            #print(file_rad == item[0], )
            if (file_rad == item[0]) and (file_angle == item[1]) and (random and (file_has_random > 0)) :
                print(file)
                
                for k, subfile in enumerate(os.listdir(os.path.join(path2scores, file))):
                    
                    if int(subfile.split('nsample')[-1].split('.npy')[0]) == item[2]:
                        
                        score_file_list.append(os.path.join(path2scores, file, subfile))
                        
    return score_file_list

def GetTemplateFiles(score_list):
    template_path = '/storage/home/adz6/group/project/datasets/data/dense_template_grid'
    
    template_list = []
    for i, score_path in enumerate(score_list):
        scores_name = score_path.split('/')[-1]
        
        angle = scores_name.split('grid_')[-1].split('_')[0]
        rad = scores_name.split('_template')[0].split('_')[-1]
        #print(angle, rad, )
        for j, template_file in enumerate(os.listdir(template_path)):
            #print(angle, rad, template_file)
            if (template_file.find(angle) > 0) and (template_file.find(rad) > 0):
                print(template_file)
                template_list.append(os.path.join(template_path, template_file))
                
    return template_list
                
def GetSignalFiles(score_list):
    signal_path = '/storage/home/adz6/group/project/datasets/data/dense_template_random'
    
    signal_list = []
    for i, score_path in enumerate(score_list):
        scores_name = score_path.split('/')[-1]
        
        angle = scores_name.split('grid_')[-1].split('_')[0]
        rad = scores_name.split('_template')[0].split('_')[-1]
        #print(angle, rad, )
        for j, signal_file in enumerate(os.listdir(signal_path)):
            #print(angle, rad, template_file)
            if (signal_file.find(angle) > 0) and (signal_file.find(rad) > 0):
                print(signal_file)
                signal_list.append(os.path.join(signal_path, signal_file))
                
    return signal_list

def MatchRatios(score_file, template_file, signal_file):
    
    score_name = score_file.split('/')[-1]
    scores = np.load(score_file)
    
    angle = score_name.split('grid_')[-1].split('_')[0]
    rad = score_name.split('_template')[0].split('_')[-1]
    nsample = int(score_name.split('_nsample')[-1].split('.npy')[0])
    nslice = int(score_name.split('_nslice')[-1].split('_')[0])
    
    
    energy_array = np.linspace(18575, 18580, 101)
    central_pitch_angle = float(angle)
    angle_array = np.linspace(central_pitch_angle - 0.005, central_pitch_angle + 0.005, 101)
    energy_grid, angle_grid = np.meshgrid(energy_array, angle_array)
    
    _template_data = mf.data.MFDataset(template_file)
    template_metadata = pd.DataFrame(_template_data.metadata)

    _signal_data = mf.data.MFDataset(signal_file)
    signal_metadata = pd.DataFrame(_signal_data.metadata)



# Plot score images (energy, angle) space

In [ ]:
#os.listdir(os.path.join(RESULTPATH, 'scores'))

In [ ]:
path2scores = os.path.join(RESULTPATH, 'scores')

rads = [1, 2, 3, 4]
samples = [8192]
angles = [86.0, 87.0, 88.0]

rads, angles, samples = np.meshgrid(rads, angles, samples)

params = {'r': rads.flatten(), 'pa': angles.flatten(), 'N': samples.flatten()}

score_list = GetScoreFiles(path2scores, params)
template_list = GetTemplateFiles(score_list)
signal_list = GetSignalFiles(score_list)


# Down select template grids

In [ ]:


energies_base = np.linspace(18575, 18580, 101)
angles_base = np.linspace(89.5 - 0.005, 89.5 + 0.005, 101)

n_point = 2

energy_grid, angle_grid = np.meshgrid(energies_base, angles_base)

#for i in range(9):

#    print(energy_grid.flatten()[np.arange(0, 101 * 101, i+1)].size)

fig = plt.figure(figsize = (13, 13))

ax = fig.add_subplot(1,1,1)
ax.plot(energy_grid.flatten()[np.arange(0, 101 * 101, n_point)], angle_grid.flatten()[np.arange(0, 101 * 101, n_point)], '.', markersize=10)

ax.set_xlabel('Energy (eV)')
ax.set_ylabel('Pitch Angle (deg)')

ax.set_title(f'Example Down-selected Grid, N = {energy_grid.flatten()[np.arange(0, 101 * 101, n_point)].size}')

name = '220111_example_down_select_grid_4'

#plt.savefig(os.path.join(PATH, 'plots', 'mayfly', name))

# animate grids

In [ ]:
sns.set_theme(context='poster')

energies_base = np.linspace(18575, 18580, 101)
angles_base = np.linspace(89.5 - 0.005, 89.5 + 0.005, 101)

energy_grid, angle_grid = np.meshgrid(energies_base, angles_base)

def animate(k):
    line.set_xdata(energy_grid.flatten()[np.arange(0, 101 * 101, k+1)])
    line.set_ydata(angle_grid.flatten()[np.arange(0, 101 * 101, k+1)])
    
    ax.set_title(k)
    
    return line,


fig = plt.figure(figsize = (13, 13))

ax = fig.add_subplot(1,1,1)

line, = ax.plot(energy_grid.flatten(), angle_grid.flatten(), '.', markersize=10)
ax.set_title(1)

#ani = matplotlib.animation.FuncAnimation(fig, animate, frames = np.arange(1, 1000, 20), interval=700)

#ani.save(os.path.join(PATH, 'plots', 'mayfly', 'test.gif', ))

# Compute Mean Matches for different track lengths, save data

In [ ]:
os.listdir(os.path.join(RESULTPATH, 'scores', ))

In [ ]:
os.listdir(os.path.join(RESULTPATH, 'scores', '220112_sens_est_dense_grid_89.5_0cm_template_scores_random_nslice2_modify_nsample'))

In [ ]:
central_pitch_angles = [84.5, 87.0, 89.5]
radial_positions = [4]
nsample_list = [1024, 1536, 2048, 2560, 3072, 3584, 4096, 4608, 5120, 5632, 6144, 6656, 7168, 7680, 8192]

random_template_list = os.listdir(os.path.join(PATH, 'datasets', 'data', 'dense_template_random'))
template_list = os.listdir(os.path.join(PATH, 'datasets', 'data', 'dense_template_grid'))

pitch_angle_grid, radial_grid = np.meshgrid(central_pitch_angles, radial_positions)

date = 220114

n_down_sample = 4000

for i, param_pair in enumerate(zip(pitch_angle_grid.flatten(), radial_grid.flatten())):
    
    scores_name = f'220114_sens_est_dense_grid_{param_pair[0]}_{param_pair[1]}cm_template_scores_random_nslice2_modify_nsample'

    angle = param_pair[0]
    radius = param_pair[1]
    
    name = f'{date}_sens_est_dense_grid_{angle}_{radius}cm_match_ratios.h5'
    path = os.path.join(PATH, 'results', 'mayfly', 'match_ratios', name)
    result_file = h5py.File(path, 'w')

    print(scores_name)
    for nsample in nsample_list:
        print(nsample)
        
        for file_name in os.listdir(os.path.join(RESULTPATH, 'scores', scores_name,)):
            if int(file_name.split('nsample')[-1].split('_')[0]) == nsample:

                scores = np.load(os.path.join(RESULTPATH, 'scores', scores_name, file_name))
                
                result_file.create_dataset(str(nsample), (n_down_sample, scores.shape[-1],))

                angles_base = np.linspace(angle - 0.005, angle + 0.005, 101)
                energies_base = np.linspace(18575, 18580, 101)

                energy_grid, angle_grid = np.meshgrid(energies_base, angles_base)
                
                for template_file in template_list:
                    if float(template_file.split('grid_')[-1].split('_')[0]) == angle and int(template_file.split('cm')[0].split('_')[-1]) == radius:
                        _template_data = mf.data.MFDataset(os.path.join(PATH, 'datasets/data', 'dense_template_grid', template_file))
                
                template_metadata = pd.DataFrame(_template_data.metadata)

                for random_template_file in random_template_list:
                    if float(random_template_file.split('grid_')[-1].split('_')[0]) == angle and int(random_template_file.split('cm')[0].split('_')[-1]) == radius:
                        _signal_data = mf.data.MFDataset(os.path.join(PATH, 'datasets/data', 'dense_template_random', random_template_file))
                        
                signal_metadata = pd.DataFrame(_signal_data.metadata)

                signal_data = _signal_data.data[:]

                signal_data = signal_data.reshape(signal_data.shape[0], 60, signal_data.shape[-1] // 60)[:, :, 0:2*nsample].reshape(signal_data.shape[0], 60 * 2 * nsample)

                ideal_scores = abs(signal_data * signal_data.conjugate()).sum(-1)

                mean_match = []
                n_grid = []
                for k in range(n_down_sample):
                    selected_template_scores = []
                    for j, pair2 in enumerate(zip(energy_grid.flatten()[np.arange(0, 101 * 101, 1 * k+1)], angle_grid.flatten()[np.arange(0, 101 * 101, k+1)])):
                        try:
                            #print(i)
                            _index = template_metadata[(template_metadata['energy'] == pair2[0]) & (template_metadata['theta_min'] == pair2[1])].index[0]
                            #print(scores[_index, random_index])
                            selected_template_scores.append(scores[_index, :])
                        except:
                            #print('Oh no!')
                            pass # some of the simulations failed so there will be zero pixels

                    selected_template_scores = np.array(selected_template_scores)
                    #print(selected_template_scores.shape)

                    match = selected_template_scores.max(0) / ideal_scores

                    result_file[str(nsample)][k, :] = match
                    n_grid.append(selected_template_scores.shape[0])
                result_file[str(nsample)].attrs.create('number-of-templates', n_grid)
                
                
    result_file.close()